In [15]:
import random
import math
import copy
import time
import logging
from typing import List, Tuple, Dict, Optional
from tabulate import tabulate

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class CuttingStockProblem:
    """Class to handle the cutting stock problem logic."""
    def __init__(self, stock_length: int, piece_lengths: List[int]):
        self.stock_length = stock_length
        self.piece_lengths = piece_lengths

    def calculate_waste(self, pattern: List[int]) -> float:
        """Calculate waste for a given cutting pattern."""
        total_length = sum(self.piece_lengths[i] * pattern[i] for i in range(len(self.piece_lengths)))
        if total_length > self.stock_length:
            return float('inf')
        return self.stock_length - total_length

    def calculate_piece_count(self, pattern: List[int]) -> int:
        """Calculate total number of pieces in a pattern."""
        return sum(pattern)

    def generate_initial_pattern(self) -> List[int]:
        """Generate a valid initial cutting pattern."""
        pattern = [0] * len(self.piece_lengths)
        remaining_length = self.stock_length
        for i in range(len(self.piece_lengths)):
            max_count = remaining_length // self.piece_lengths[i]
            pattern[i] = random.randint(0, max_count)
            remaining_length -= pattern[i] * self.piece_lengths[i]
        return pattern

class Algorithm:
    """Base class for optimization algorithms."""
    @staticmethod
    def simulated_annealing(csp: CuttingStockProblem, iterations: int = 1000, initial_temp: float = 1000) \
            -> Tuple[List[int], float, int]:
        current_pattern = csp.generate_initial_pattern()
        current_waste = csp.calculate_waste(current_pattern)
        best_pattern = current_pattern[:]
        best_waste = current_waste
        temp = initial_temp
        loop_count = 0

        for _ in range(iterations):
            loop_count += 1
            new_pattern = current_pattern[:]
            idx = random.randint(0, len(csp.piece_lengths) - 1)
            change = random.choice([-1, 1])
            new_pattern[idx] = max(0, new_pattern[idx] + change)

            new_waste = csp.calculate_waste(new_pattern)
            if new_waste == float('inf'):
                continue

            delta = new_waste - current_waste
            if delta < 0 or random.random() < math.exp(-delta / temp):
                current_pattern = new_pattern[:]
                current_waste = new_waste

            if current_waste < best_waste or (current_waste == best_waste and csp.calculate_piece_count(current_pattern) < csp.calculate_piece_count(best_pattern)):
                best_pattern = current_pattern[:]
                best_waste = current_waste

            temp *= 0.99

        return best_pattern, best_waste, loop_count

    @staticmethod
    def hill_climbing(csp: CuttingStockProblem, iterations: int = 100) -> Tuple[List[int], float, int]:
        current_pattern = csp.generate_initial_pattern()
        current_waste = csp.calculate_waste(current_pattern)
        loop_count = 0

        for _ in range(iterations):
            loop_count += 1
            neighbors = []
            for i in range(len(csp.piece_lengths)):
                for change in [-1, 1]:
                    neighbor = current_pattern[:]
                    neighbor[i] = max(0, neighbor[i] + change)
                    waste = csp.calculate_waste(neighbor)
                    if waste != float('inf'):
                        neighbors.append((neighbor, waste))

            if not neighbors:
                break

            best_neighbor, best_waste = min(neighbors, key=lambda x: (x[1], csp.calculate_piece_count(x[0])))
            if best_waste > current_waste or (best_waste == current_waste and csp.calculate_piece_count(best_neighbor) >= csp.calculate_piece_count(current_pattern)):
                break
            current_pattern, current_waste = best_neighbor, best_waste

        return current_pattern, current_waste, loop_count

    @staticmethod
    def genetic_algorithm(csp: CuttingStockProblem, population_size: int = 50, generations: int = 100) \
            -> Tuple[List[int], float, int]:
        def create_individual():
            return csp.generate_initial_pattern()

        population = [create_individual() for _ in range(population_size)]
        loop_count = 0

        for _ in range(generations):
            loop_count += 1
            fitness = [(ind, csp.calculate_waste(ind)) for ind in population]
            population = sorted(fitness, key=lambda x: (x[1], csp.calculate_piece_count(x[0])))
            new_population = [ind for ind, _ in population[:population_size // 2]]

            while len(new_population) < population_size:
                parent1, parent2 = random.choice(population[:population_size // 2])[0], random.choice(population[:population_size // 2])[0]
                crossover_point = random.randint(0, len(csp.piece_lengths))
                child = parent1[:crossover_point] + parent2[crossover_point:]
                if random.random() < 0.1:
                    idx = random.randint(0, len(csp.piece_lengths) - 1)
                    child[idx] = max(0, child[idx] + random.choice([-1, 1]))
                if csp.calculate_waste(child) != float('inf'):
                    new_population.append(child)
                else:
                    new_population.append(create_individual())

            population = new_population

        best_individual = min(population, key=lambda x: (csp.calculate_waste(x), csp.calculate_piece_count(x)))
        return best_individual, csp.calculate_waste(best_individual), loop_count

class Agent:
    """Agent class to encapsulate algorithm execution."""
    ALGORITHM_MAP = {
        "SA": Algorithm.simulated_annealing,
        "HC": Algorithm.hill_climbing,
        "GA": Algorithm.genetic_algorithm
    }

    def __init__(self, algorithm: str):
        algorithm = algorithm.upper()
        if algorithm not in self.ALGORITHM_MAP:
            raise ValueError(f"Invalid algorithm: {algorithm}. Choose from {list(self.ALGORITHM_MAP.keys())}")
        self.algorithm = algorithm

    def solve(self, csp: CuttingStockProblem) -> Tuple[List[int], float, int, float]:
        start_time = time.time()
        pattern, waste, loops = self.ALGORITHM_MAP[self.algorithm](csp)
        runtime = time.time() - start_time
        logging.info(f"Agent {self.algorithm} completed with waste: {waste}, loops: {loops}, runtime: {runtime:.4f}s")
        return pattern, waste, loops, runtime

class CuttingStockSolver:
    """Main solver class for the cutting stock problem."""
    def __init__(self, stock_length: int, piece_lengths: List[int]):
        self.algorithms = ["SA", "HC", "GA"]
        self.stock_length = stock_length
        self.piece_lengths = piece_lengths
        self.csp = CuttingStockProblem(stock_length, piece_lengths)

    def solve(self, num_agents: int, algorithms: List[str], approach_type: str) -> Tuple[List[int], float, int, float, List[Tuple]]:
        algorithms = [alg.upper() for alg in algorithms]
        if not algorithms:
            raise ValueError("No algorithms provided.")
        if not all(alg in self.algorithms for alg in algorithms):
            raise ValueError(f"Invalid algorithms: {algorithms}. Choose from {self.algorithms}")

        assigned_algorithms = []
        for i in range(num_agents):
            assigned_algorithms.append(algorithms[i % len(algorithms)])

        agents = [Agent(alg) for alg in assigned_algorithms]
        best_pattern, best_waste = None, float('inf')
        total_loops, total_runtime = 0, 0
        agent_results = []

        if approach_type == "1":
            for agent in agents:
                pattern, waste, loops, runtime = agent.solve(self.csp)
                agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                total_loops += loops
                total_runtime += runtime
                if waste < best_waste or (waste == best_waste and self.csp.calculate_piece_count(pattern) < self.csp.calculate_piece_count(best_pattern or pattern)):
                    best_pattern, best_waste = pattern[:], waste

            for _ in range(3):
                for agent in agents:
                    pattern, waste, loops, runtime = agent.solve(self.csp)
                    agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                    total_loops += loops
                    total_runtime += runtime
                    if waste < best_waste or (waste == best_waste and self.csp.calculate_piece_count(pattern) < self.csp.calculate_piece_count(best_pattern or pattern)):
                        best_pattern, best_waste = pattern[:], waste
        else:
            for agent in agents:
                pattern, waste, loops, runtime = agent.solve(self.csp)
                agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                total_loops += loops
                total_runtime += runtime
                if waste < best_waste or (waste == best_waste and self.csp.calculate_piece_count(pattern) < self.csp.calculate_piece_count(best_pattern or pattern)):
                    best_pattern, best_waste = pattern[:], waste

        return best_pattern, best_waste, total_loops, total_runtime, agent_results

    def run_experiments(self):
        """Run experiments testing all combinations of variables."""
        headers = ["Exp ID", "Agent Count", "Algorithms", "Approach", "Cutting Pattern", "Waste", "Piece Count", "Loop Count", "Runtime"]
        table_data = []
        exp_id = 1

        # Define variables
        agent_counts = [3, 6]
        approaches = ["1", "2"]  # 1: Collaborative, 2: Hyper-heuristic
        algorithm_combinations = [
            ["SA"], ["HC"], ["GA"],
            ["SA", "HC"], ["SA", "GA"], ["HC", "GA"],
            ["SA", "HC", "GA"]
        ]

        # Run experiments for all combinations
        for num_agents in agent_counts:
            for approach in approaches:
                for algorithms in algorithm_combinations:
                    pattern, waste, total_loops, total_runtime, _ = self.solve(num_agents, algorithms, approach)
                    pattern_str = ", ".join([f"{self.piece_lengths[j]}m x {count}" for j, count in enumerate(pattern)])
                    piece_count = self.csp.calculate_piece_count(pattern)
                    table_data.append([
                        exp_id,
                        num_agents,
                        ", ".join(algorithms),
                        "Collaborative" if approach == "1" else "Hyper",
                        pattern_str,
                        waste,
                        piece_count,
                        total_loops,
                        f"{total_runtime:.4f}s"
                    ])
                    exp_id += 1

        # Display all results
        print(f"\nStock Length: {self.stock_length}")
        print(f"Piece Lengths: {self.piece_lengths}")
        print("\nAll Combinations Experiment Results")
        print(tabulate(table_data, headers=headers, tablefmt="grid"))

        # Find best results (lowest waste, then lowest runtime, then lowest piece count)
        valid_results = [row for row in table_data if row[5] != float('inf')]
        if valid_results:
            best_waste = min(row[5] for row in valid_results)
            lowest_waste_results = [row for row in valid_results if row[5] == best_waste]
            # Convert runtime strings to float for comparison
            best_runtime = min(float(row[8][:-1]) for row in lowest_waste_results)
            lowest_runtime_results = [row for row in lowest_waste_results if float(row[8][:-1]) == best_runtime]
            # Find lowest piece count
            best_piece_count = min(row[6] for row in lowest_runtime_results)
            best_results = [row for row in lowest_runtime_results if row[6] == best_piece_count]
        else:
            best_results = []

        print("\nBest Results (Lowest Waste, Lowest Runtime, Lowest Piece Count)")
        print(tabulate(best_results, headers=headers, tablefmt="grid"))

def main():
    """Main function."""
    try:
        # Get user input for stock length and piece lengths
        stock_length = int(input("Enter stock length (e.g., 100): "))
        piece_lengths_input = input("Enter piece lengths separated by commas (e.g., 10,15,20): ")
        piece_lengths = [int(x) for x in piece_lengths_input.split(',')]

        # Validate inputs
        if stock_length <= 0:
            raise ValueError("Stock length must be positive.")
        if not piece_lengths:
            raise ValueError("At least one piece length must be provided.")
        if any(pl <= 0 for pl in piece_lengths):
            raise ValueError("Piece lengths must be positive.")
        if any(pl > stock_length for pl in piece_lengths):
            raise ValueError("Piece lengths cannot exceed stock length.")

        solver = CuttingStockSolver(stock_length, piece_lengths)
        solver.run_experiments()
    except ValueError as e:
        logging.error(f"Input error: {e}")
        print(f"Error: {e}")
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        print("An unexpected error occurred. Please try again.")

if __name__ == "__main__":
    main()

Enter stock length (e.g., 100): 101
Enter piece lengths separated by commas (e.g., 10,15,20): 11,17,23

Stock Length: 101
Piece Lengths: [11, 17, 23]

All Combinations Experiment Results
+----------+---------------+--------------+---------------+---------------------------+---------+---------------+--------------+-----------+
|   Exp ID |   Agent Count | Algorithms   | Approach      | Cutting Pattern           |   Waste |   Piece Count |   Loop Count | Runtime   |
+==========+===============+==============+===============+===========================+=========+===============+==============+===========+
|        1 |             3 | SA           | Collaborative | 11m x 5, 17m x 0, 23m x 2 |       0 |             7 |        12000 | 0.0520s   |
+----------+---------------+--------------+---------------+---------------------------+---------+---------------+--------------+-----------+
|        2 |             3 | HC           | Collaborative | 11m x 5, 17m x 0, 23m x 2 |       0 |           

In [18]:
import random
import math
import copy
import time
import logging
from typing import List, Tuple, Dict, Optional
from tabulate import tabulate

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class CuttingStockProblem:
    """Class to handle the cutting stock problem logic."""
    def __init__(self, stock_length: int, piece_lengths: List[int]):
        self.stock_length = stock_length
        self.piece_lengths = piece_lengths

    def calculate_waste(self, pattern: List[int]) -> float:
        """Calculate waste for a given cutting pattern."""
        total_length = sum(self.piece_lengths[i] * pattern[i] for i in range(len(self.piece_lengths)))
        if total_length > self.stock_length:
            return float('inf')
        return self.stock_length - total_length

    def calculate_piece_count(self, pattern: List[int]) -> int:
        """Calculate total number of pieces in a pattern."""
        return sum(pattern)

    def generate_initial_pattern(self) -> List[int]:
        """Generate a valid initial cutting pattern."""
        pattern = [0] * len(self.piece_lengths)
        remaining_length = self.stock_length
        for i in range(len(self.piece_lengths)):
            max_count = remaining_length // self.piece_lengths[i]
            pattern[i] = random.randint(0, max_count)
            remaining_length -= pattern[i] * self.piece_lengths[i]
        return pattern

class Algorithm:
    """Base class for optimization algorithms."""
    @staticmethod
    def simulated_annealing(csp: CuttingStockProblem, iterations: int = 1000, initial_temp: float = 1000) \
            -> Tuple[List[int], float, int]:
        current_pattern = csp.generate_initial_pattern()
        current_waste = csp.calculate_waste(current_pattern)
        best_pattern = current_pattern[:]
        best_waste = current_waste
        temp = initial_temp
        loop_count = 0

        for _ in range(iterations):
            loop_count += 1
            new_pattern = current_pattern[:]
            idx = random.randint(0, len(csp.piece_lengths) - 1)
            change = random.choice([-1, 1])
            new_pattern[idx] = max(0, new_pattern[idx] + change)

            new_waste = csp.calculate_waste(new_pattern)
            if new_waste == float('inf'):
                continue

            delta = new_waste - current_waste
            if delta < 0 or random.random() < math.exp(-delta / temp):
                current_pattern = new_pattern[:]
                current_waste = new_waste

            if current_waste < best_waste or (current_waste == best_waste and csp.calculate_piece_count(current_pattern) < csp.calculate_piece_count(best_pattern)):
                best_pattern = current_pattern[:]
                best_waste = current_waste

            temp *= 0.99

        return best_pattern, best_waste, loop_count

    @staticmethod
    def hill_climbing(csp: CuttingStockProblem, iterations: int = 100) -> Tuple[List[int], float, int]:
        current_pattern = csp.generate_initial_pattern()
        current_waste = csp.calculate_waste(current_pattern)
        loop_count = 0

        for _ in range(iterations):
            loop_count += 1
            neighbors = []
            for i in range(len(csp.piece_lengths)):
                for change in [-1, 1]:
                    neighbor = current_pattern[:]
                    neighbor[i] = max(0, neighbor[i] + change)
                    waste = csp.calculate_waste(neighbor)
                    if waste != float('inf'):
                        neighbors.append((neighbor, waste))

            if not neighbors:
                break

            best_neighbor, best_waste = min(neighbors, key=lambda x: (x[1], csp.calculate_piece_count(x[0])))
            if best_waste > current_waste or (best_waste == current_waste and csp.calculate_piece_count(best_neighbor) >= csp.calculate_piece_count(current_pattern)):
                break
            current_pattern, current_waste = best_neighbor, best_waste

        return current_pattern, current_waste, loop_count

    @staticmethod
    def genetic_algorithm(csp: CuttingStockProblem, population_size: int = 50, generations: int = 100) \
            -> Tuple[List[int], float, int]:
        def create_individual():
            return csp.generate_initial_pattern()

        population = [create_individual() for _ in range(population_size)]
        loop_count = 0

        for _ in range(generations):
            loop_count += 1
            fitness = [(ind, csp.calculate_waste(ind)) for ind in population]
            population = sorted(fitness, key=lambda x: (x[1], csp.calculate_piece_count(x[0])))
            new_population = [ind for ind, _ in population[:population_size // 2]]

            while len(new_population) < population_size:
                parent1, parent2 = random.choice(population[:population_size // 2])[0], random.choice(population[:population_size // 2])[0]
                crossover_point = random.randint(0, len(csp.piece_lengths))
                child = parent1[:crossover_point] + parent2[crossover_point:]
                if random.random() < 0.1:
                    idx = random.randint(0, len(csp.piece_lengths) - 1)
                    child[idx] = max(0, child[idx] + random.choice([-1, 1]))
                if csp.calculate_waste(child) != float('inf'):
                    new_population.append(child)
                else:
                    new_population.append(create_individual())

            population = new_population

        best_individual = min(population, key=lambda x: (csp.calculate_waste(x), csp.calculate_piece_count(x)))
        return best_individual, csp.calculate_waste(best_individual), loop_count

class Agent:
    """Agent class to encapsulate algorithm execution."""
    ALGORITHM_MAP = {
        "SA": Algorithm.simulated_annealing,
        "HC": Algorithm.hill_climbing,
        "GA": Algorithm.genetic_algorithm
    }

    def __init__(self, algorithm: str):
        algorithm = algorithm.upper()
        if algorithm not in self.ALGORITHM_MAP:
            raise ValueError(f"Invalid algorithm: {algorithm}. Choose from {list(self.ALGORITHM_MAP.keys())}")
        self.algorithm = algorithm

    def solve(self, csp: CuttingStockProblem) -> Tuple[List[int], float, int, float]:
        start_time = time.time()
        pattern, waste, loops = self.ALGORITHM_MAP[self.algorithm](csp)
        runtime = time.time() - start_time
        logging.info(f"Agent {self.algorithm} completed with waste: {waste}, loops: {loops}, runtime: {runtime:.4f}s")
        return pattern, waste, loops, runtime

class CuttingStockSolver:
    """Main solver class for the cutting stock problem."""
    def __init__(self, stock_length: int, piece_lengths: List[int]):
        self.algorithms = ["SA", "HC", "GA"]
        self.stock_length = stock_length
        self.piece_lengths = piece_lengths
        self.csp = CuttingStockProblem(stock_length, piece_lengths)

    def solve(self, num_agents: int, algorithms: List[str], approach_type: str) -> Tuple[List[int], float, int, float, List[Tuple]]:
        algorithms = [alg.upper() for alg in algorithms]
        if not algorithms:
            raise ValueError("No algorithms provided.")
        if not all(alg in self.algorithms for alg in algorithms):
            raise ValueError(f"Invalid algorithms: {algorithms}. Choose from {self.algorithms}")

        assigned_algorithms = []
        for i in range(num_agents):
            assigned_algorithms.append(algorithms[i % len(algorithms)])

        agents = [Agent(alg) for alg in assigned_algorithms]
        best_pattern, best_waste = None, float('inf')
        total_loops, total_runtime = 0, 0
        agent_results = []

        if approach_type == "1":
            for agent in agents:
                pattern, waste, loops, runtime = agent.solve(self.csp)
                agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                total_loops += loops
                total_runtime += runtime
                if waste < best_waste or (waste == best_waste and self.csp.calculate_piece_count(pattern) < self.csp.calculate_piece_count(best_pattern or pattern)):
                    best_pattern, best_waste = pattern[:], waste

            for _ in range(3):
                for agent in agents:
                    pattern, waste, loops, runtime = agent.solve(self.csp)
                    agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                    total_loops += loops
                    total_runtime += runtime
                    if waste < best_waste or (waste == best_waste and self.csp.calculate_piece_count(pattern) < self.csp.calculate_piece_count(best_pattern or pattern)):
                        best_pattern, best_waste = pattern[:], waste
        else:
            for agent in agents:
                pattern, waste, loops, runtime = agent.solve(self.csp)
                agent_results.append((agent.algorithm, pattern, waste, loops, runtime))
                total_loops += loops
                total_runtime += runtime
                if waste < best_waste or (waste == best_waste and self.csp.calculate_piece_count(pattern) < self.csp.calculate_piece_count(best_pattern or pattern)):
                    best_pattern, best_waste = pattern[:], waste

        return best_pattern, best_waste, total_loops, total_runtime, agent_results

    def run_single_experiment(self, exp_id: int) -> List:
        """Run a single experiment with user-specified parameters."""
        headers = ["Exp ID", "Agent Count", "Algorithms", "Approach", "Cutting Pattern", "Waste", "Piece Count", "Loop Count", "Runtime"]

        # Get agent count
        print("\nSelect number of agents (1-9):")
        while True:
            try:
                num_agents = int(input("Your choice: "))
                if 1 <= num_agents <= 9:
                    break
                else:
                    print("Please enter a number between 1 and 9.")
            except ValueError:
                print("Please enter a valid number.")

        # Get algorithm combination
        print("\nSelect algorithm combination:")
        algorithm_options = [
            ["SA"], ["HC"], ["GA"],
            ["SA", "HC"], ["SA", "GA"], ["HC", "GA"],
            ["SA", "HC", "GA"]
        ]
        for i, alg in enumerate(algorithm_options, 1):
            print(f"{i}: {', '.join(alg)}")
        while True:
            try:
                alg_choice = int(input("Your choice (1-7): "))
                if 1 <= alg_choice <= 7:
                    algorithms = algorithm_options[alg_choice - 1]
                    break
                else:
                    print("Please enter a number between 1 and 7.")
            except ValueError:
                print("Please enter a valid number.")

        # Get approach type
        print("\nSelect approach type:")
        print("1: Collaborative")
        print("2: Hyper-heuristic")
        while True:
            try:
                approach_choice = int(input("Your choice (1-2): "))
                if approach_choice == 1:
                    approach_type = "1"
                    approach_name = "Collaborative"
                    break
                elif approach_choice == 2:
                    approach_type = "2"
                    approach_name = "Hyper"
                    break
                else:
                    print("Please enter 1 or 2.")
            except ValueError:
                print("Please enter a valid number.")

        # Run the experiment
        pattern, waste, total_loops, total_runtime, _ = self.solve(num_agents, algorithms, approach_type)
        pattern_str = ", ".join([f"{self.piece_lengths[j]}m x {count}" for j, count in enumerate(pattern)])
        piece_count = self.csp.calculate_piece_count(pattern)

        # Prepare result
        result = [
            exp_id,
            num_agents,
            ", ".join(algorithms),
            approach_name,
            pattern_str,
            waste,
            piece_count,
            total_loops,
            f"{total_runtime:.4f}s"
        ]

        # Display result
        print(f"\nStock Length: {self.stock_length}")
        print(f"Piece Lengths: {self.piece_lengths}")
        print("\nExperiment Result")
        print(tabulate([result], headers=headers, tablefmt="grid"))

        return result

def run_experiment_cycle():
    """Run a single experiment cycle, including input collection."""
    try:
        # Get user input for stock length and piece lengths
        stock_length = int(input("Enter stock length (e.g., 100): "))
        piece_lengths_input = input("Enter piece lengths separated by commas (e.g., 10,15,20): ")
        piece_lengths = [int(x) for x in piece_lengths_input.split(',')]

        # Validate inputs
        if stock_length <= 0:
            raise ValueError("Stock length must be positive.")
        if not piece_lengths:
            raise ValueError("At least one piece length must be provided.")
        if any(pl <= 0 for pl in piece_lengths):
            raise ValueError("Piece lengths must be positive.")
        if any(pl > stock_length for pl in piece_lengths):
            raise ValueError("Piece lengths cannot exceed stock length.")

        solver = CuttingStockSolver(stock_length, piece_lengths)
        solver.run_single_experiment(1)
        return True
    except ValueError as e:
        logging.error(f"Input error: {e}")
        print(f"Error: {e}")
        return False
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        print("An unexpected error occurred. Please try again.")
        return False

def main():
    """Main function."""
    while True:
        if not run_experiment_cycle():
            print("Program terminated due to an error.")
            break
        continue_choice = input("\nDo you want to run another experiment? (y/n): ").lower()
        if continue_choice != 'y':
            print("Program terminated.")
            break

if __name__ == "__main__":
    main()

Enter stock length (e.g., 100): 176
Enter piece lengths separated by commas (e.g., 10,15,20): 12,23

Select number of agents (1-9):
Your choice: 3

Select algorithm combination:
1: SA
2: HC
3: GA
4: SA, HC
5: SA, GA
6: HC, GA
7: SA, HC, GA
Your choice (1-7): 3

Select approach type:
1: Collaborative
2: Hyper-heuristic
Your choice (1-2): 1

Stock Length: 176
Piece Lengths: [12, 23]

Experiment Result
+----------+---------------+--------------+---------------+-------------------+---------+---------------+--------------+-----------+
|   Exp ID |   Agent Count | Algorithms   | Approach      | Cutting Pattern   |   Waste |   Piece Count |   Loop Count | Runtime   |
+==========+===============+==============+===============+===================+=========+===============+==============+===========+
|        1 |             3 | GA           | Collaborative | 12m x 7, 23m x 4  |       0 |            11 |         1200 | 0.1556s   |
+----------+---------------+--------------+---------------+------